## NLP Sentiment Analysis

### Introduction
The purpose of this notebook is to create a proof-of-concept for sentiment analysis using existing NLP analysis library. The target corpus come from news articles that are published on the web, New York Times. Through this POC, it's desired to provide an example that qualitatively assesses the overall sentiment of article and how well the assessment aligns with human interpretation. A stretch goal of this POC includes automatically generating a summary of the article, which also gets qualitatively evaluated by human reader.

### Method
For the NLP analytic library, the following library is used. () 
The target corpus comes from NYT website, and the total number of articles is 3. Each article has a different level sentiment.
The human reader or interpretator is the author of this POC.

### Code
The below block of code loads essential libraries for this project.

In [38]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

The setiment analyzing tool is imported from VADER library. Each article is read and stored as a list of strings--each element of the list representing a paragraph of that article--and mapped to a specific key.

In [39]:
# Spins up sentiment analyzer
analyzer = SentimentIntensityAnalyzer()
articles = ['article_1', 'article_2', 'article_3']
article_map = {
    'article_1': '',
    'article_2': '',
    'article_3': '',
}
for article in articles:
    with open(article + '.txt', 'r') as f:
        for line in f:
            article_map[article] += line
    article_map[article] = article_map[article].split('\n\n')
    print(article_map[article])

['The U.S. labor market has been less resilient than was initially believed. On Wednesday, the Labor Department said that the economy had added 818,000 fewer jobs than it had previously reported for the 12 months that ended in March.', 'The number means employers had overstated job growth by about 28 percent per month, especially in industries like hospitality and professional services. The downward revision adds to growing evidence of a weakening job market: The unemployment rate, though still relatively low, ticked up to 4.3 percent last month.', 'This adjusted number is an initial estimate of an annual revision, in which monthly employment figures from the Labor Department are reconciled with more accurate state unemployment reports. This year’s revision was unusually large: Over the previous decade, annual updates added or subtracted around 173,000 jobs, on average.', '“We’ve known that things on net were probably moving gradually in the wrong direction,” said Guy Berger, director 

The below blocks of codes breaks down each article into either paragraphs and sentences for sentiment analysis. The VADER sentiment analyzer works better at the sentence level, as the New York Times articles in Economics departments tend to have minimal sentimental bias, and the results from the sentence-level analyses yielded more neutral sentiments.

In [40]:
def analyzeArticlesByParagraph():
    for article in article_map.keys():
        paragraphSentiments = 0.0
        for paragraph in article_map[article]:
            vs = analyzer.polarity_scores(paragraph)
            paragraphSentiments += vs["compound"]
        print("AVERAGE SENTIMENT OF PARAGRAPHS: \t" + str(round(paragraphSentiments / len(article), 4)))
        print("\t")

def analyzeArticlesBySentence():
    for article in article_map.keys():
        print("ARTICLE: {}".format(article))
        averageSetimentScore = 0.0
        setimentScoreSum = 0.0
        for paragraph in article_map[article]:
            sentenceSentiments = 0.0
            sentences = paragraph.split(".")
            for sentence in sentences:
                vaderScore = analyzer.polarity_scores(sentence)
                sentenceSentiments += vaderScore["compound"]
            setimentScoreSum += round(sentenceSentiments / len(sentences), 4)

        print("AVERAGE SENTIMENT OF ARTICLE: \t" + str(round(setimentScoreSum / len(article), 4)))
        print("\t")



In [41]:
print("BY PARAGRAPHS:")
analyzeArticlesByParagraph()
print("BY SENTENCES:")
analyzeArticlesBySentence()

BY PARAGRAPHS:
AVERAGE SENTIMENT OF PARAGRAPHS: 	-0.1051
	
AVERAGE SENTIMENT OF PARAGRAPHS: 	0.7995
	
AVERAGE SENTIMENT OF PARAGRAPHS: 	-0.0746
	
BY SENTENCES:
ARTICLE: article_1
AVERAGE SENTIMENT OF ARTICLE: 	-0.0332
	
ARTICLE: article_2
AVERAGE SENTIMENT OF ARTICLE: 	0.2669
	
ARTICLE: article_3
AVERAGE SENTIMENT OF ARTICLE: 	-0.0139
	


## Text Summarization
For summarizing text, a natural language processor tool, NLTK, is used. The preprocesses for the analysis includes taking stop words into account and weighing each word by its appearence frequency within the corpus. The articles are summarized and outputed to the current working directory.

In [42]:
import nltk               
from nltk.corpus import stopwords                       
from nltk.tokenize import word_tokenize, sent_tokenize 

In [43]:
def summarize(txt):
    stopWords = set(stopwords.words("english")) 
    words = word_tokenize(txt)
    freqTable = dict()                 
    for word in words:               
        word = word.lower()                 
        if word in stopWords:                 
            continue                  
        if word in freqTable:                       
            freqTable[word] += 1            
        else:          
            freqTable[word] = 1  
    sentences = sent_tokenize(txt)                 
    sentenceValue = dict()
    for sentence in sentences:               
        for word, freq in freqTable.items():        
            if word in sentence.lower():                   
                if sentence in sentenceValue:                                 
                    sentenceValue[sentence] += freq                       
                else:                       
                    sentenceValue[sentence] = freq                    

    sumValues = 0                        
    for sentence in sentenceValue:              
        sumValues += sentenceValue[sentence]

    average = int(sumValues / len(sentenceValue))
    summary = ''
    for sentence in sentences:               
        if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)): # Adjust summary length here by increasing/decreasing the number             
            summary += " " + sentence
    return summary




In [44]:
for article in article_map.keys():
    out_file = open('{0}_summary.txt'.format(article), 'w')
    out_file.write(summarize(' '.join(article_map[article])))
    out_file.close()